### This example demonstrates the insertion of a human-defined classifier into the XCSF population set.

In [1]:
from __future__ import annotations

import json
from typing import Final

import xcsf

xcs = xcsf.XCS(8, 1, 2)  # (x_dim, y_dim, n_actions)
xcs.condition("hyperrectangle")
xcs.action("integer")
xcs.prediction("nlms_linear")

classifier: Final[dict] = {
    "error": 10,  # each of these properties are optional
    "fitness": 1.01,
    "accuracy": 2,
    "set_size": 100,
    "numerosity": 2,
    "experience": 3,
    "time": 3,
    "samples_seen": 2,
    "samples_matched": 1,
    "condition": {
        "type": "hyperrectangle",
        "center": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        "spread": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        "mutation": [0.2],  # this parameter still self-adapts
    },
    "action": {"type": "integer", "action": 1, "mutation": [0.28]}
    # prediction is absent and therefore initialised as normal
}

json_str: str = json.dumps(classifier)  # dictionary to JSON

# The json_insert_cl() function can be used to insert a single new classifier
# into the population. The new classifier is initialised with a random
# condition, action, prediction, and then any supplied properties overwrite
# these values. This means that all properties are optional. If the population
# set numerosity exceeds xcs.POP_SIZE after inserting the rule, the standard
# roulette wheel deletion mechanism will be invoked to maintain the population
# limit.
xcs.json_insert_cl(json_str)  # insert in [P]

### Get the population as JSON and print

In [2]:
pop_str: str = xcs.json(True, True, True)
print(pop_str)

{
	"classifiers":	[{
			"error":	10,
			"fitness":	1.01,
			"accuracy":	2.2204460492503131e-16,
			"set_size":	100,
			"numerosity":	2,
			"experience":	3,
			"time":	3,
			"samples_seen":	2,
			"samples_matched":	1,
			"current_match":	false,
			"current_action":	0,
			"current_prediction":	[0],
			"condition":	{
				"type":	"hyperrectangle",
				"center":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"spread":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"mutation":	[0.2]
			},
			"action":	{
				"type":	"integer",
				"action":	1,
				"mutation":	[0.28]
			},
			"prediction":	{
				"type":	"nlms_linear",
				"weights":	[1, 0, 0, 0, 0, 0, 0, 0, 0],
				"eta":	0.0037507443022723646,
				"mutation":	[0.60528699252168372]
			}
		}]
}
